# [1193. Monthly Transactions I](https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Transactions

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+</pre>
id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ("approved", "declined").
 

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Transactions table:
<pre>+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |
+------+---------+----------+--------+------------+</pre>
Output: 
<pre>+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+</pre>

In [1]:
# pandas schema

import pandas as pd

data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'],
        [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype(
    {'id': 'Int64', 'country': 'object', 'state': 'object', 'amount': 'Int64', 'trans_date': 'datetime64[ns]'})

In [2]:
# to spark dataframe

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

transactions_df = spark.createDataFrame(transactions)
transactions_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/09 13:47:43 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/09 13:47:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 13:47:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-------+--------+------+-------------------+
|id |country|state   |amount|trans_date         |
+---+-------+--------+------+-------------------+
|121|US     |approved|1000  |2018-12-18 00:00:00|
|122|US     |declined|2000  |2018-12-19 00:00:00|
|123|US     |approved|2000  |2019-01-01 00:00:00|
|124|DE     |approved|2000  |2019-01-07 00:00:00|
+---+-------+--------+------+-------------------+


In [3]:
# solving in spark dataframe
import pyspark.sql.functions as F

transactions_df.withColumn('month', F.substring('trans_date', 0, 7)) \
    .groupBy('month', 'country') \
    .agg(
    F.count('id').alias('trans_count') \
    , F.sum(F.when(F.col('state') == 'approved', 1).otherwise(0)).alias('approved_count') \
    , F.sum('amount').alias('trans_total_amount') \
    , F.sum(F.when(F.col('state') == 'approved', F.col('amount')).otherwise(0)).alias('approved_total_amount') \
    ) \
    .show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+


In [4]:
# solving in spark SQL

transactions_df.createOrReplaceTempView('transactions')

spark.sql('''
select 
    left(trans_date,7) as month,
    country,
    count(id) as trans_count,
    sum(case when state='approved' then 1 else 0 end) as approved_count,
    sum(amount) as trans_total_amount,
    sum(case when state='approved' then amount else 0 end) as approved_total_amount
from transactions
group by month, country;
''').show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+


In [5]:
spark.stop()